# Welcome to OT as Plate filler script 

start with choosing the number of plates that you want to run and then continue on to setting up the deck. 

FIND SCRIPT ON ROBOT OT028 IN FOLDER Vilhelms_sandbox

In [5]:
## insures that robot is correctly initialized
!systemctl stop opentrons-robot-server

'systemctl' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# import the script you wnat to run 
import sys
import pandas as pd
import os, shutil
# from datetime import datetime
from IPython.display import Markdown as md
# ## start email and camera functions
import subprocess
import smtplib
import datetime
from datetime import date
import os
from time import sleep
# # Import opentrons.execute
import opentrons.execute


ModuleNotFoundError: No module named 'opentrons'

# Initiate Robot 

In [ ]:

# import datetime
from opentrons import types
from time import sleep


def camera(name_of_image, protocol): 
    """ Take picture with the OT camera, runs bash command 
    input: users name of image
    output: image in directory 
    """

    # camerapath = '/dev/video0'
    # camerapath = '/dev/video1'
    camerapath = '/dev/video2'
    subprocess.check_call(['ffmpeg', '-y', '-f', 'video4linux2', '-s',
                       '640x480', '-i', camerapath, '-ss', '0:0:1', 
                       '-frames', '1', os.path.abspath('.') + '/pictures/' + name_of_image +'.jpg']) 


    print('Cammera on')

protocol = opentrons.execute.get_protocol_api('2.11')

protocol.home()


In [ ]:
protocol = opentrons.execute.get_protocol_api('2.11')

##### RUN CODE ON OPENTRONS
p300tips = [protocol.load_labware('opentrons_96_tiprack_300ul', 11)]
p300 = protocol.load_instrument('p300_multi', 'right', tip_racks=p300tips)

fake_plate = protocol.load_labware('agilent_1_reservoir_290ml', 6)

microplate_type = 'nest_96_wellplate_200ul_flat'
microplates = [protocol.load_labware(microplate_type, s) for s in ['4']]

hight = 20
center_location = fake_plate['A1'].top()
combe_move_right = center_location.move(types.Point(x=50, y=0, z=hight))
combe_move_left = center_location.move(types.Point(x=-50, y=0, z=hight))

# Start Run

In [ ]:
import json
import numpy as np

## Turn on lights while running run
protocol.set_rail_lights(True)

#saved image
source = 'save-image-to-path'
image_name = 'name-of-image'

#detect file
file_path = 'detect-python-file-path'
#arguments = ["--source", source,'--save-txt'] 


# Take picture of the liquid level
p300.pick_up_tip(p300tips[0]['A1'])
p300.aspirate(150, microplates[0]['A1'].bottom(1))
p300.move_to(combe_move_right)
camera(f'liquid_level', protocol)
image_path_ll = os.path.join(source, 'liquid_level') #path of the liquid level image
arguments_txt = ["--source", image_path_ll,'--save-txt']# important to keep the '--save-txt'

completed_process = subprocess.run(["python", file_path] + arguments_txt, capture_output=True, text=True)
output = completed_process.stdout

In [ ]:
def find_key_value_by_keyword(dictionary, keyword):
    key_value_dict = {key: value for key, value in dictionary.items() if keyword in key and value}
    key_value_dict = list(key_value_dict.values())
    key_value = list()
    for value in key_value_dict[0]:
        key_value.append(value['bbox'])
    return key_value

In [ ]:
def detect(output1):
    bbox_dict1 = output1
    bbox_dict = json.loads(bbox_dict1)
    tips = find_key_value_by_keyword(bbox_dict, 'tip')
    if len(tips) != 8:
        print(len(tips))
        print('error')
        p300.pause("error") 
        liquids = [None]   
    else:
        liquids = find_key_value_by_keyword(bbox_dict, 'liquid')
    return tips,liquids

In [ ]:
def save_threshold(boxes1, boxes2):
    matched_boxes = []
    liquid_level_all = []

    for box1 in boxes1:
        min_distance = float('inf')
        matched_box = None
     
        for box2 in boxes2:
            distance = abs(box1[0] - box2[0])
            if distance < min_distance:
                min_distance = distance
                matched_box = box2

        if matched_box is not None:
            liquid_level = abs(box1[1] / matched_box[1])
            matched_boxes.append((box1, matched_box))
            liquid_level_all.append(liquid_level)
    ### not use numpy
    #total = sum(arr)
    #length = len(arr)
    #average = total / length
    liquid_level_mean = np.mean(liquid_level_all)
    print(liquid_level_mean)
    
    return liquid_level_mean

In [ ]:
def match_boxes(boxes1, boxes2, threshold):
    matched_boxes = []

    for box1 in boxes1:
        min_distance = float('inf')
        matched_box = None

        for box2 in boxes2:
            distance = abs(box1[0] - box2[0])
            if distance < min_distance:
                min_distance = distance
                matched_box = box2

        if matched_box is not None:
            liquid_level = abs(box1[1] / matched_box[1])
            # 预警
            if 0.8 * threshold < liquid_level < 1.1*threshold:
                print('no warning')
            else:
                print(f'Warning: Liquid level is below the threshold! Current level: {liquid_level}')
            matched_boxes.append((box1, matched_box))

    return

In [ ]:
tips,liquids = detect(output)
threshold = save_threshold(tips,liquids)
p300.dispense(160, microplates[0]['A1'].bottom(1))
p300.return_tip(p300tips[0]['A1'])
protocol.home()

In [ ]:
## LOOP START
for i in range(20):
    source_name = f'{image_name}_{i+1}'
    source_path = f'{source}{source_name}'
    arguments = ["--source", source_path,'--save-txt']
    
    #take tips
    p300.pick_up_tip(p300tips[0]['A1'])
    #go to camera
    p300.aspirate(150, microplates[0]['A1'].bottom(1))
    p300.move_to(combe_move_right)
    camera(source_name, protocol) #Overwrite the previous image file
    completed_process = subprocess.run(["python", file_path] + arguments, capture_output=True, text=True)
    output = completed_process.stdout
    tips,liquids = detect(output)
    match_boxes(tips,liquids,threshold)
    p300.dispense(160, microplates[0]['A1'].bottom(1))
    p300.return_tip(p300tips[0]['A1'])
    protocol.home()
    
## Turn on lights while running run
protocol.set_rail_lights(False)

print('Your protocol is finished well done!')

# MY PLAN
# tips no water
# tips with water 
# tips with Red water 
# No tips on pipet
# tipsEmptyResdual

# tips with X(color) water 

HOW TO EXTRACT CODE FROM OT: 
- OPEN WSL OR UBUNTU ON YOUR COMPUTER
- WRITE: sudo scp -i ot2_ssh_key -r root@10.199.253.233:/var/lib/jupyter/notebooks/Vilhelms_sandbox/pictures /mnt/c/Intel/
- GO TO Intel folder on C drive on your computer and find the file there. 

# with liquid level detection